# Load Cell Browser browser into Vitessce

This notebooks shows you how to use two scripts, part of the python-vitessce libaray, which allow you to take an existing project, published in https://cells.ucsc.edu/ and either:
1. Convert it into format that is supported by Vitessce and load it into Vitessce using the [automatic view config generation option](http://localhost:3001/docs/default-config-json/),
or:
2. Get the Vitessce view config JSON directly.

For both of these options, the dataset that you choose to convert needs to be a valid project, accessible from here: https://cells.ucsc.edu/, with configuration available in here: https://github.com/ucscGenomeBrowser/cellbrowser-confs

Each of the scripts needs the name of that project as an input. For example to convert this project: https://cells.ucsc.edu/?ds=adultPancreas, you will neeed to pass `adultPancreas` as a project name for either of the scripts.

## 1. Covert UCSC Cell Browser project to a format that is supported by Vitessce
#### Ouput:
An Anndata-Zarr object, saved in `vitessce-files/project_name.out.adata.zarr` directory by default on your computer.
Check the documentation for more information on each of the parameters available for this function.

#### How to load the output in Vitessce:
A) Upload the Anndata-Zarr object, returned by the script, to a publicly available HTTP store
B) Paste the URL to the Anndata-Zarr object in http://vitessce.io/#?edit=true to generate a view config (see the docs about [automatic view config generation option](http://localhost:3001/docs/default-config-json/)).

In [4]:
# Import the relevant function
from vitessce import (convert_cell_browser_project_to_anndata)

In [5]:
# Example run, coverting "adultPancreas" project:
convert_cell_browser_project_to_anndata(project_name="adultPancreas")

Converting CellBrowser config for project adultPancreas to Anndata-Zarr object and saving it to vitessce-files
Successfully fetched configuration: https://cells.ucsc.edu/adultPancreas/dataset.json.
CellBrowser config is valid. Proceeding further with conversion.
Successfully downloaded expression matrix https://cells.ucsc.edu/adultPancreas/exprMatrix.tsv.gz.
Loading expression matrix into Anndata object ...
This dataset uses the format identifier|symbol for the ad.obs gene names (e.g. “ENSG0123123.3|HOX3”). We are keeping only the symbol.
Successfully loaded expression matrix into Anndata object.
save_intermediate_object is False. The current version of the Anndata object is not saved.
Adding cell metadata to Anndata object ...
Successfully downloaded metadata meta.tsv.
save_intermediate_object is False. The current version of the Anndata object is not saved.
Successful extraction of the following coordinates and URLS: {'X_tsne': 'tMinusSNE.coords.tsv.gz'}
Adding X_tsne to Anndata obje

## 2. Covert UCSC Cell Browser project to a Vitessce view config
#### Ouputs:
- An Anndata object, saved in `vitessce-files/project_name/out.adata.h5ad` by default on your computer.
- A JSON, representing the Vitessce view config for the project.
Check the documentation for more information on each of the parameters available for this function.

#### How to load the output in Vitessce:
A) Upload the Anndata object, returned by the script, to a publicly available HTTP store.
B) Paste the Vitessce view config JSON in [Vitessce](http://vitessce.io/#?edit=true). In `files`, under `url` add the URL of the Anndata object obtained from the previous step.

In [2]:
# Import the relevant function
from vitessce import (convert_cellbrowser_project_to_vitessce_config)

In [3]:
# Example run, coverting "adultPancreas" project:
convert_cellbrowser_project_to_vitessce_config(project_name="adultPancreas")

Converting CellBrowser config for project adultPancreas to Vitessce view config
Successfully fetched configuration: https://cells.ucsc.edu/adultPancreas/dataset.json.
CellBrowser config is valid. Proceeding further with conversion.
Successfully downloaded expression matrix https://cells.ucsc.edu/adultPancreas/exprMatrix.tsv.gz.
Loading expression matrix into Anndata object ...
This dataset uses the format identifier|symbol for the ad.obs gene names (e.g. “ENSG0123123.3|HOX3”). We are keeping only the symbol.
Successfully loaded expression matrix into Anndata object.
save_intermediate_object is False. The current version of the Anndata object is not saved.
Adding cell metadata to Anndata object ...
Successfully downloaded metadata meta.tsv.
save_intermediate_object is False. The current version of the Anndata object is not saved.
Successful extraction of the following coordinates and URLS: {'X_tsne': 'tMinusSNE.coords.tsv.gz'}
Adding X_tsne to Anndata object ...
X_tsne successfully adde